In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def prever_numeros(sorteios_df, dim_tempo_df, data_referencia=None):
    # Define a data de hoje
    if data_referencia is None:
        hoje = pd.to_datetime(datetime.today().date())
    else:
        hoje = pd.to_datetime(data_referencia)

    # Define os dias da semana em ordem (0 = segunda, ..., 6 = domingo)
    dias_sorteio = {'terça-feira': 1, 'quinta-feira': 3, 'sábado': 5}

    # Converte colunas para datetime se necessário
    sorteios_df['data_sorteio'] = pd.to_datetime(sorteios_df['data_sorteio'])
    dim_tempo_df['data'] = pd.to_datetime(dim_tempo_df['data'])

    # Adiciona coluna de dia da semana no sorteios_df com base na dimensão tempo
    sorteios_df = sorteios_df.merge(dim_tempo_df[['data', 'dia_semana']],
                                    left_on='data_sorteio', right_on='data',
                                    how='left').drop(columns=['data'])

    # Define os próximos dias de sorteio a partir de hoje
    proximos_sorteios = []
    for i in range(1, 8):
        dia = hoje + timedelta(days=i)
        dia_semana = dia.weekday()  # segunda = 0, ..., domingo = 6
        if dia_semana in dias_sorteio.values():
            nome_dia = list(dias_sorteio.keys())[list(dias_sorteio.values()).index(dia_semana)]
            proximos_sorteios.append((dia, nome_dia))
        if len(proximos_sorteios) == 3:
            break

    # Previsões por dia da semana
    previsoes = {}

    for dia, nome_dia in proximos_sorteios:
        # Filtra o histórico apenas para aquele dia da semana
        historico_dia = sorteios_df[sorteios_df['dia_semana'] == nome_dia.lower()]

        # Extrai os números sorteados e conta a frequência
        todos_numeros = historico_dia['num_sorteados'].explode()
        frequencia = todos_numeros.value_counts()

        # Seleciona os 15 mais frequentes
        top_15 = frequencia.head(15).index.tolist()

        previsoes[nome_dia] = top_15

    return previsoes


In [ ]:
# Exemplo de DataFrame de sorteios
sorteios_df = pd.DataFrame({
    'data_sorteio': ['2024-05-07', '2024-05-09', '2024-05-11'],  # etc.
    'num_sorteados': [[1, 2, 3, 4, 5, 6], [2, 3, 5, 7, 9, 10], [1, 4, 5, 6, 8, 12]]
})

# Exemplo de DataFrame de dimensão tempo
dim_tempo_df = pd.DataFrame({
    'data': pd.date_range(start='2024-01-01', end='2025-12-31'),
})
dim_tempo_df['dia_semana'] = dim_tempo_df['data'].dt.day_name(locale='pt_BR').str.lower()

# Chamada da função
resultado = prever_numeros(sorteios_df, dim_tempo_df, data_referencia='2025-05-25')

# Exibe resultado
for dia, numeros in resultado.items():
    print(f"{dia.capitalize()}: {numeros}")
